## Ingesting PDF

In [ ]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [2]:
local_path = "../pdf_files/owner_manual_p283-p300.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

/home/estaciolima/pdf_reader/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from IPython.display import display, Markdown
def view_text_in_markdown(page_content):
    display(Markdown(page_content))

## Vector Embeddings

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
!ollama pull mistral

In [ ]:
!ollama list

In [ ]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [4]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [35]:
chunks

[Document(metadata={'source': '../pdf_files/owner_manual_p283-p300.pdf'}, page_content='282 IN CASE OF EMERGENCY\n\nIN CASE OF EMERGENCY\n\nHAZARD WARNING FLASHERS\n\nThe Hazard Warning Flashers button is located on the upper switch bank just below the radio.\n\nHazard Warning Flashers Button\n\nNOTE:\n\nIf your vehicle is equipped with a 12-inch Uconnect display, the Hazard Warning Flashers button is located above the display.\n\nHazard Warning Flashers Button with 12-inch display\n\nNOTE:\n\nIf your vehicle is equipped with a 14.5-inch Uconnect display, the Hazard Warning Flashers button is located to the left of the display.\n\nHazard Warning Flashers Button with 14.5-inch display'),
 Document(metadata={'source': '../pdf_files/owner_manual_p283-p300.pdf'}, page_content='Hazard Warning Flashers Button with 14.5-inch display\n\nPush the button to turn on the Hazard Warning Flashers. When the button is activated, all directional turn signals will flash on and off to warn oncoming, traf

In [5]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 60/60 [00:10<00:00,  5.76it/s]


## Retrieval

In [7]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [8]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [9]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [10]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
retrieve_docs = retriever.invoke('Where is located the hazard flashers button?') # get relevant documents

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 14.89it/s]


In [16]:
retrieve_docs

[Document(metadata={'source': '../pdf_files/owner_manual_p283-p300.pdf'}, page_content='282 IN CASE OF EMERGENCY\n\nIN CASE OF EMERGENCY\n\nHAZARD WARNING FLASHERS\n\nThe Hazard Warning Flashers button is located on the upper switch bank just below the radio.\n\nHazard Warning Flashers Button\n\nNOTE:\n\nIf your vehicle is equipped with a 12-inch Uconnect display, the Hazard Warning Flashers button is located above the display.\n\nHazard Warning Flashers Button with 12-inch display\n\nNOTE:\n\nIf your vehicle is equipped with a 14.5-inch Uconnect display, the Hazard Warning Flashers button is located to the left of the display.\n\nHazard Warning Flashers Button with 14.5-inch display'),
 Document(metadata={'source': '../pdf_files/owner_manual_p283-p300.pdf'}, page_content='Hazard Warning Flashers Button with 14.5-inch display\n\nPush the button to turn on the Hazard Warning Flashers. When the button is activated, all directional turn signals will flash on and off to warn oncoming, traf

In [17]:
len(retrieve_docs)

8

In [21]:
view_text_in_markdown(retrieve_docs[1].page_content)

Hazard Warning Flashers Button with 14.5-inch display

Push the button to turn on the Hazard Warning Flashers. When the button is activated, all directional turn signals will flash on and off to warn oncoming, traffic of an emergency. Push the button a second time ‘to turn off the Hazard Warning Flashers.

This is an emergency warning system and it should not be used when the vehicle is in motion. Use only when your vehicle is disabled or signaling a safety hazard warning for other motorists.

When leaving the vehicle to seek assistance, the

Hazard Warning Flashers will continue to operate even though the ignition is placed in the OFF position.

NOTE: With extended use the Hazard Warning Flashers may wear down your battery.

ASSIST AND SOS SYSTEM — IF EQUIPPED

‘A0703000080US

In [23]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
resposta = chain.invoke("Where is located the hazard flashers button?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.18it/s]


In [26]:
view_text_in_markdown(resposta)

 The Hazard Warning Flashers button can be located on the upper switch bank just below the radio, above the 12-inch Uconnect display, or to the left of the 14.5-inch Uconnect display depending on the vehicle's equipment.

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()